# Agentic workflow

Fron Andrew Ng
> Today, we mostly use LLMs in zero-shot mode, prompting a model to generate final output token by token without revising its work. This is akin to asking someone to compose an essay from start to finish, typing straight through with no backspacing allowed, and expecting a high-quality result. Despite the difficulty, LLMs do amazingly well at this task! 

With an agent workflow, however, we can ask the LLM to iterate over a document many times. For example, it might take a sequence of steps such as:

- Plan an outline.
- Decide what, if any, web searches are needed to gather more information.
- Write a first draft.
- Read over the first draft to spot unjustified arguments or extraneous information.
- Revise the draft taking into account any weaknesses spotted.

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
from utils import get_completion, read_file,AZURE_OPENAI_API_GPT_35_MODEL

model = AZURE_OPENAI_API_GPT_35_MODEL

In [2]:
constraints_and_example = """
<constraints_and_example>
    Constraints:
    1. Please give headline and list down the 3 key points as bullet points. I'm particularly interested in key points and any significant technological advancements or implications discussed.
    2. If there are any fund raising details, please include the amount, the investors and valuation. If no fund raising info, just don't mention it."
    3. Please DON't include the words "Headline" and "Key points" in the output.
    4. Don't include any duplicate info.
    
    Example output:
    Apple Introduces Private Cloud Compute for Enhanced AI Privacy.
    • Apple has unveiled Private Cloud Compute (PCC), a cloud intelligence system designed to enhance the privacy of AI processing in the cloud. The system ensures user data sent to PCC isn't accessible to anyone, not even Apple, extending the security and privacy of Apple devices into the cloud.
    • The PCC system is built with custom Apple silicon and a secure operating system. It is designed to fulfill user requests using complex machine learning models while maintaining privacy. The system maintains user data on PCC nodes only until the response is returned and does not retain any data afterward.
    • To ensure the integrity of the system, Apple is making software images of every production build of PCC publicly available for scrutiny by the security research community. The company will also release a PCC Virtual Research Environment and a subset of the security-critical PCC source code to aid research.
</constraints_and_example>
"""
INITIAL_SUMMARIZE_SYSTEM_PROMPT = """
You are a tech journalist writing a summary of a tech article. You will be provided with a tech article and you need to summarize it with key points and any significant technological advancements.
"""

INITIAL_SUMMARIZE_USER_PROMPT = """Your task is to carefully read a source text and the summary of the text.
{constraints_and_example}
<SOURCE_TEXT>
{text}
</SOURCE_TEXT>

Your output:
"""

In [3]:


article = read_file("data/article.txt")
user_input = INITIAL_SUMMARIZE_USER_PROMPT.format(constraints_and_example= constraints_and_example, text= article) 
# print(user_input)
response = get_completion(user_input, INITIAL_SUMMARIZE_SYSTEM_PROMPT, model=model)
print(response)

==================LLM gpt-35-turbo response======================
LazyGraphRAG: Setting a New Standard for GraphRAG Quality and Cost
• LazyGraphRAG introduces a new approach to graph-enabled RAG that requires no prior summarization of source data, offering scalability in cost and quality.
• LazyGraphRAG blends the advantages of vector RAG and Graph RAG, showing strong performance across cost-quality spectrum.
• LazyGraphRAG's answer quality is state-of-the-art, outperforming competing methods on both local and global queries with varying relevance test budgets.


In [4]:

user_input = INITIAL_SUMMARIZE_USER_PROMPT.format(constraints_and_example= constraints_and_example, text= article)
initial_response = get_completion(user_input, INITIAL_SUMMARIZE_SYSTEM_PROMPT, model=model)
print(initial_response)

==================LLM gpt-35-turbo response======================
LazyGraphRAG: Setting a New Standard for GraphRAG Quality and Cost
• LazyGraphRAG introduces a new approach to graph-enabled RAG that requires no prior summarization of source data, offering scalability in cost and quality.
• LazyGraphRAG blends the advantages of vector RAG and Graph RAG, showing strong performance across cost-quality spectrum.
• LazyGraphRAG's answer quality is state-of-the-art, outperforming competing methods on both local and global queries with varying relevance test budgets.


### Step 2. Reflection

In [5]:

# Reflection
REFLECTION_SUMMARIZE_SYSTEM_PROMPT = """You are a tech journalist writing a summary of a tech article. You will be provided with a source text and its summary, and your goal is to improve the summary."""
REFLECTION_SUMMARIZE_USER_PROMPT = """Your task is to carefully read a source text and the summary of the text, and then improve the summary with accuracy and constraints. Then give constructive criticism and helpful suggestions for improving the summary.
<SOURCE_TEXT>
{text}
</SOURCE_TEXT>

The summary of the indicated part, delimited below by <SUMMARY> and </SUMMARY>, is as follows:
<SUMMARY>
{summary}
</SUMMARY>

When writing suggestions, pay attention to whether there are ways to improve the summary's:\n\
(i) accuracy (by correcting errors of addition, missummary, omission, or unsummaried text),\n\
(ii) fluency (by applying grammar, spelling and punctuation rules, and ensuring there are no unnecessary repetitions),\n\
(iii) terminology (by ensuring terminology use is consistent and reflects the source text domain;).\n\
{constraints_and_example}
Write a list of specific, helpful and constructive suggestions for improving the summary.
Each suggestion should address one specific part of the summary.

Please put your suggestions in <Reflection> tags.
"""

reflection_input = REFLECTION_SUMMARIZE_USER_PROMPT.format(constraints_and_example= constraints_and_example, text= article, summary= initial_response)

reflection_response = get_completion(reflection_input, REFLECTION_SUMMARIZE_SYSTEM_PROMPT, model=model)
print(reflection_response)

==================LLM gpt-35-turbo response======================
<Reflection>
1. The summary should mention that LazyGraphRAG is a new approach that combines the benefits of vector RAG and Graph RAG without the need for prior data summarization, emphasizing its scalability in terms of cost and quality.
2. Include details about LazyGraphRAG's performance across the cost-quality spectrum, highlighting its superiority over competing methods in both local and global queries.
3. Emphasize the state-of-the-art answer quality of LazyGraphRAG and its ability to outperform other methods with varying relevance test budgets.
</Reflection>


### Review

In [6]:
FINAL_SUMMARIZE_SYSTEM_PROMPT = INITIAL_SUMMARIZE_SYSTEM_PROMPT
FINAL_SUMMARIZE_USER_PROMPT = """
Your task is to carefully read, then improve a summary, taking into
account a set of expert suggestions and constructive criticisms. Below, the source text, initial summary, and expert suggestions are provided.

The source text is below, delimited by XML tags <SOURCE_TEXT> and </SOURCE_TEXT>, and the part that has been summarized
is delimited by <SUMMARIZE_THIS> and </SUMMARIZE_THIS> within the source text. You can use the rest of the source text
as context, but need to provide a summary only of the part indicated by <SUMMARIZE_THIS> and </SUMMARIZE_THIS>.

<SOURCE_TEXT>
{text}
</SOURCE_TEXT>

The summary of the indicated part, delimited below by <SUMMARIZE_THIS> and </SUMMARIZE_THIS>, is as follows:
<SUMMARIZE_THIS>
{summary}
</SUMMARIZE_THIS>

The expert summarys of the indicated part, delimited below by <EXPERT_SUGGESTIONS> and </EXPERT_SUGGESTIONS>, is as follows:
<EXPERT_SUGGESTIONS>
{reflection}
</EXPERT_SUGGESTIONS>

Taking into account the expert suggestions rewrite the summary to improve it, paying attention
to whether there are ways to improve the summary's
(i) accuracy (by correcting errors of addition, missummary, omission, or unsummaried text),\n\
(ii) fluency (by applying grammar, spelling and punctuation rules, and ensuring there are no unnecessary repetitions),\n\
(iii) style (by ensuring the summaries is in a enaging mananer of social media post),\n\
(iv) terminology (by ensuring terminology use is consistent and reflects the source text domain;).\n\

{constraints_and_example}

Please put the final output in <final_summary> tags.
Your output:
"""
reviewed_input = FINAL_SUMMARIZE_USER_PROMPT.format(constraints_and_example= constraints_and_example, text= article, summary= initial_response, reflection= reflection_response)

reviewed_response = get_completion(reviewed_input, FINAL_SUMMARIZE_SYSTEM_PROMPT, model=model)
print("Final output:")
print(reviewed_response)

==================LLM gpt-35-turbo response======================
Final output:
<final_summary>
LazyGraphRAG: Revolutionizing GraphRAG Efficiency and Quality
• LazyGraphRAG introduces a novel approach to graph-enabled RAG that eliminates the need for prior data summarization, showcasing scalability in both cost and quality.
• The system combines the strengths of vector RAG and Graph RAG, demonstrating superior performance across the cost-quality spectrum compared to competing methods.
• LazyGraphRAG achieves state-of-the-art answer quality, surpassing other methods in both local and global queries with varying relevance test budgets.
</final_summary>


### Congrats!

You've done all the lab works. Happy prompting!